Implementa algoritmo Held-Karp per risolvere il problema del comesso viaggiattore in modo ottimale

In [1]:
import sys
sys.path.append('..')

from WorldParser import WorldParser
import networkx as nx
from HeldKarp import HeldKarp


des_file = """
MAZE: "mylevel", ' '
FLAGS:premapped
GEOMETRY:center,center

MAP
|||||||||||||
|...........|
|..|........|
|...........|
|.......|...|
|.....|.....|
|...|....|..|
|||||||||||||
ENDMAP

BRANCH:(1,1,1,1),(0,0,0,0)
GOLD:1,(1,2)
GOLD:1,(2,1)
GOLD:1,(7,4)
GOLD:1,(9,4)
GOLD:1,(11,5)
GOLD:1,(6,1)

"""

starting_position = (1,1)

In [2]:
def calculate_distances(G, points):
    """
    Restituisce un dizionare dove per ogni punto calcola la distanza minima con tutti gli altri punti.
    """
    
    distances = {}
    for point in points:
        lengths = nx.single_source_dijkstra_path_length(G, point)
        distances[point] = {other: lengths[other] for other in points if other in lengths}
    return distances

In [3]:
parser = WorldParser(des_file)
gold_positions = parser.extract_gold_positions()
world_graph = parser.extract_world_graph()

In [4]:
heldKarp = HeldKarp(world_graph, gold_positions, starting_position)
heldKarp.TSP()

20